In [1]:
import torch
import torch.nn as nn
import sys
import os
from torchvision import datasets,transforms
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import torchvision
from loguru import logger
import argparse
import torchvision.models as models

/global/cfs/projectdirs/m2136/backup/conda/envs/pytorch1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. CNN model

In [5]:
epoch = 500
batch_size = 128

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv2 = nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(32*56*56, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64,5)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0),-1)
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        
        return x

In [7]:
class myDataset(Dataset):
    def __init__(self, data_path):
        n = 224
        transform = transforms.Compose([transforms.Resize((n,n)), transforms.ToTensor()])
        self.data = datasets.ImageFolder(f'{data_path}',transform)

    def __getitem__(self, index):
        x,y = self.data[index]
        y = torch.tensor(y)
        x = x.to(device)
        y = y.to(device)

        return x,y

    def __len__(self):
        self.num = len(self.data.targets)
        return self.num

In [19]:
resnet = models.resnet50(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_features, 5)
resnet = resnet.to(device)

# 2. data

In [10]:
train_path = f'/pscratch/sd/z/zhangtao/storm/mpc/key_paper/training'
test_path  = f'/pscratch/sd/z/zhangtao/storm/mpc/key_paper/test'
n = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.Resize((n,n)), transforms.ToTensor()])

In [6]:
train_data = datasets.ImageFolder(f'{train_path}',transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

n_train = len(train_data.targets)

# 3. train

In [7]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for e in range(epoch):
    train_loss = 0.0
    for i,data in enumerate(train_loader):
        x,y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        out = model(x)    
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    print(f'epoch={e}, loss = {train_loss/n_train}')

In [8]:
model = torch.load('../saved_models/cnn')

In [8]:
test_path = f'/pscratch/sd/z/zhangtao/storm/mpc/key_paper/test/'
test_data = myDataset(test_path)
test_loader = DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [20]:
for data in test_loader:
    with torch.no_grad():
        x,y = data
        out = resnet(x)
        
        print(y.shape)
        print(out.shape)
    sys.exit()
    
    

torch.Size([128])
torch.Size([128, 5])


SystemExit: 